## Haber Tahmin 

- `best_meta.json` yüklenir ve **TF-IDF → SVD → Model** hattı otomatik kurulur.
- Girdi metni eğitimdeki **aynı temizleme** fonksiyonundan geçer.
- Model `predict_proba` destekliyse en yüksek olasılık; değilse **decision margin** gösterilir.
- Komut satırından girilen metinler için hızlı demo.


In [2]:
# Haber Tahmin (meta uyumlu) — TF-IDF → SVD (train'de SMOTE) PIPELINE

import json, re, string, sys, numpy as np
from joblib import load

# --- Stopwords & Stemmer (eğitimle uyumlu temizlik) ---
def get_turkish_stopwords():
    try:
        import nltk
        from nltk.corpus import stopwords
        try:
            nltk.data.find('corpora/stopwords')
        except LookupError:
            nltk.download('stopwords')
        return set(stopwords.words('turkish'))
    except Exception:
        # yedek kısa liste
        return set(['ve','veya','ama','fakat','ancak','de','da','ile','gibi','çok','daha','hem','ki','mi','mı','mu','mü',
                    'için','çünkü','ya','yani','ise','o','bu','şu','ben','sen','biz','siz','onlar','bir','birlikte','olan',
                    'olarak','kadar','önce','sonra','şöyle','böyle','hiç','her','hepsi','çünkü','neden','niçin','ne','nasıl'])

TURKISH_STOPWORDS = get_turkish_stopwords()

try:
    from TurkishStemmer import TurkishStemmer
    stemmer = TurkishStemmer()
    print("TurkishStemmer yüklendi.")
except Exception:
    stemmer = None
    print("Uyarı: TurkishStemmer bulunamadı, kök bulma atlanacak.")

EXTRA_PUNCT = "’‘“”–—…•·ªº°"
PUNCT_TABLE = str.maketrans('', '', string.punctuation + EXTRA_PUNCT)

def clean_text_tr(text: str) -> str:
    if not isinstance(text, str):
        return ""
    t = text.casefold()
    t = re.sub(r'https?://\S+|www\.\S+', ' ', t)   # URL
    t = re.sub(r'<.*?>', ' ', t)                   # HTML
    t = t.translate(PUNCT_TABLE)                   # noktalama
    t = re.sub(r'\d+', ' ', t)                     # rakamlar
    t = re.sub(r'\s+', ' ', t).strip()             # boşluk
    words = []
    for w in t.split():
        if w not in TURKISH_STOPWORDS:
            words.append(stemmer.stem(w) if stemmer else w)
    return ' '.join(words)

# --- Meta & öğeleri yükle ---
try:
    with open('best_meta.json', 'r', encoding='utf-8') as f:
        meta = json.load(f)
except FileNotFoundError:
    print("Hata: 'best_meta.json' yok. Önce modeli kaydeden hücreyi çalıştırın.")
    sys.exit(1)

model      = load(meta['model_file'])
vectorizer = load(meta['vectorizer_file'])
svd        = load(meta['svd_file']) if meta.get('svd_file') else None
classes_   = meta.get('classes_') or getattr(model, 'classes_', None)

print(f"Yüklendi → {meta.get('model_name','Model')} | {meta.get('representation','TFIDF')}")

def vectorize_texts(raw_texts):
    cleaned = [clean_text_tr(t) for t in raw_texts]
    X = vectorizer.transform(cleaned)
    if svd is not None:
        X = svd.transform(X)
    return X

# --- Etkileşimli tahmin ---
print("\n--- Yeni Haber Metnini Sınıflandırma ---")
print("Metin girin (çıkmak için 'q'):")

while True:
    try:
        new_news = input("\nYeni haber metni: ").strip()
    except EOFError:
        break
    if new_news.lower() == 'q':
        print("Çıkılıyor.")
        break
    if not new_news:
        print("Metin boş olamaz.")
        continue

    X = vectorize_texts([new_news])
    pred = model.predict(X)[0]

    # Olasılık varsa göster; yoksa karar marjını göster
    extra = ""
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(X)[0]
        top_idx = int(np.argmax(proba))
        extra = f" (p={float(proba[top_idx]):.2f})"
    elif hasattr(model, "decision_function"):
        scores = model.decision_function(X)
        if scores.ndim == 1:  # binary
            extra = f" (margin={float(abs(scores[0])):.2f})"
        else:
            top_idx = int(np.argmax(scores[0]))
            extra = f" (margin={float(scores[0][top_idx]):.2f})"

    print(f"\nTahmin Edilen Kategori: {str(pred).upper()}{extra}")
    if classes_ is not None:
        # ilk 3 sınıfı da göster (olasılık ya da skor)
        try:
            if hasattr(model, "predict_proba"):
                vec = model.predict_proba(X)[0]
            else:
                vec = model.decision_function(X)[0]
            order = np.argsort(vec)[::-1]
            topk = [(classes_[i], float(vec[i])) for i in order[:3]]
            print("En yakın 3 sınıf:", ", ".join([f"{c}:{v:.2f}" for c, v in topk]))
        except Exception:
            pass
    print("-" * 50)


TurkishStemmer yüklendi.
Yüklendi → LinearSVC_bal | TFIDF->SVD (SMOTE train)

--- Yeni Haber Metnini Sınıflandırma ---
Metin girin (çıkmak için 'q'):



Yeni haber metni:  Günün ilk yarısında düşüş eğiliminde hareket eden BIST 100 endeksi, saat 13.00 itibarıyla önceki kapanışa göre 18,45 puan ve yüzde 0,17 azalışla 10.810,48 puana geriledi.  Toplam işlem hacmi 59,4 milyar lira oldu. Bankacılık endeksi yüzde 1,39, holding endeksi de yüzde 0,09 değer kazandı.  Sektör endekslerinden en çok kazanan yüzde 1,88 ile finansal kiralama faktoring, en fazla düşen yüzde 1,56 ile turizm oldu.  Küresel piyasalarda, yatırımcıların odağı tarım dışı istihdam verilerine çevrilirken, güne pozitif başlayan ve 10.900 puanı test eden BIST 100 endeksi bu seviyeden gelen satışlarla düşüşe geçerek günün ilk yarısını negatif bir seyirle tamamladı.



Tahmin Edilen Kategori: EKONOMI (margin=2.05)
En yakın 3 sınıf: ekonomi:2.05, spor:-1.33, sağlık:-1.36
--------------------------------------------------



Yeni haber metni:  A Milli Futbol Takımı'nın İspanya ile oynayacağı 2026 FIFA Dünya Kupası Elemeleri E Grubu maçını İngiliz hakem Michael Oliver yönetecek. Türkiye Futbol Federasyonundan yapılan açıklamaya göre, MEDAŞ Konya Büyükşehir Stadı'nda 7 Eylül Pazar günü oynanacak ve saat 21.45'te başlayacak müsabakada Oliver'ın yardımcılıklarını Stuart Burt ve James Mainwaring yapacak. Karşılaşmanın dördüncü hakemi Christopher Kavanagh olacak. Maçta Jarred Gavan Gillett VAR, Darren England ise AVAR görevini üstlenecek.



Tahmin Edilen Kategori: SPOR (margin=2.07)
En yakın 3 sınıf: spor:2.07, türkiye:-1.54, siyaset:-1.73
--------------------------------------------------



Yeni haber metni:  q


Çıkılıyor.
